In [1]:
import torch
import torch.nn as nn
import numpy as np

# Prints the number of trainable params in a model.
def get_number_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    print(params,'trainable parameters')


# initialize weights with mean=0 and std= .02 like in DCGAN
def initialize_weights(m,debug=False):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find('Linear') != -1 or classname.find('BatchNorm') != -1:
        nn.init.constant_(m.bias.data, 0)
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if debug:
            print('init',classname)
    else:
        if debug:
            print('noinit',classname)
        
# See page 15 of the Wavegan paper https://openreview.net/pdf?id=ByMVTsR5KQ        
class WaveGenerator(nn.Module):
    def __init__(self,d=64, c=1 ,inplace=True,ngpu=1):
        super(WaveGenerator, self).__init__()
        self.ngpu = ngpu
        self.d=d # model size
        self.c=c # = 1 in the paper
        self.dense1= nn.Linear(100, 256*self.d)
        self.padding=11
        self.seq = nn.Sequential(
            # input is dense(Z), going into a convolution
            nn.ReLU(inplace), #out (n,16,16d)
            nn.ConvTranspose1d( 16*self.d, self.d * 8, 25, 4, self.padding,1, bias=True), # (25,16d,4d) | (n,64,8d)
            nn.ReLU(inplace), #no batch norm
         
            nn.ConvTranspose1d(self.d * 8, self.d * 4, 25, 4, self.padding,1, bias=True),#(25, 8d, 4d)| (n, 256, 4d)
            nn.ReLU(inplace),
        
            nn.ConvTranspose1d( self.d * 4,self.d * 2, 25, 4, self.padding,1, bias=True),#(25, 4d, 2d) | (n, 1024, 2d)
            nn.ReLU(inplace),
      
            nn.ConvTranspose1d( self.d * 2, self.d, 25, 4, self.padding,1, bias=True), #(25, 2d, d) | (n, 4096, d)
            nn.ReLU(inplace),
            nn.ConvTranspose1d( self.d, self.c, 25, 4, self.padding,1, bias=True),#(25, d, c) | (n, 16384, c)
            nn.Tanh() # as suggested       
        )

    def forward(self, x):
        #input (n,100)
        x=self.dense1(x) # output (n,256*d)
        x=torch.reshape(x, (-1,16*self.d,16)) # output (n,16,16d)
        
        return self.seq(x) #(n, 16384, c), c=1

    
# See page 15 of the Wavegan paper https://openreview.net/pdf?id=ByMVTsR5KQ        
class WaveDiscriminator(nn.Module):
    def __init__(self,d=64,length=16384, c=1,inplace=True,ngpu=1):
        super(WaveDiscriminator, self).__init__()
        self.ngpu = ngpu
        self.d=d # model size
        self.c=c # = 1 in the paper
        self.padding=11
        self.length=length
        leak=0.2
        
        
        self.dense= nn.Linear(256*self.d, 1)
        self.seq = nn.Sequential(
            # input is audio or WaveGenerator(z)
            nn.Conv1d( self.c, self.d, 25, 4, self.padding, bias=True), #(n,4096,d)
            nn.LeakyReLU(leak,inplace=inplace),
            
            nn.Conv1d( self.d, 2*self.d, 25, 4, self.padding, bias=True),  #(n,1024,2d)
            nn.LeakyReLU(leak,inplace=inplace),
            
            nn.Conv1d( self.d * 2, self.d * 4, 25, 4, self.padding, bias=True),  #(n,256,4d)
            nn.LeakyReLU(leak,inplace=inplace),
            
            nn.Conv1d( self.d * 4, self.d * 8, 25, 4, self.padding, bias=True),  #(n,64,8d)
            nn.LeakyReLU(leak,inplace=inplace),
            
            nn.Conv1d( self.d * 8, self.d * 16, 25, 4, self.padding, bias=True),  #(n,16,16d)
            nn.LeakyReLU(leak,inplace=inplace)
        )
               

    def forward(self, x):
        x=self.seq(x)
        x=torch.reshape(x, (-1,256*self.d)) 
        #print(x.shape)
        return self.dense(x) 
    

    



def testing():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Create the generator
    waveG = WaveGenerator().to(device)
    #initialize wights
    waveG.apply(initialize_weights)
    print(waveG)

    # Create the Discriminator
    waveD = WaveDiscriminator().to(device)
    #initialize wights
    waveD.apply(initialize_weights)
    print(waveD)


    N, in_channels, L = 8, 1, 16384
    noise_dim = 100
    x = torch.randn((N, in_channels,L))
    waveD = WaveDiscriminator()
    print(waveD(x).shape)
    waveG = WaveGenerator(d=64,c=1)
    z = torch.randn((N, 100))
    print(waveG(z).shape)
    print(waveD(waveG(z)))
    
testing()

WaveGenerator(
  (dense1): Linear(in_features=100, out_features=16384, bias=True)
  (seq): Sequential(
    (0): ReLU(inplace=True)
    (1): ConvTranspose1d(1024, 512, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (2): ReLU(inplace=True)
    (3): ConvTranspose1d(512, 256, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (4): ReLU(inplace=True)
    (5): ConvTranspose1d(256, 128, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (6): ReLU(inplace=True)
    (7): ConvTranspose1d(128, 64, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (8): ReLU(inplace=True)
    (9): ConvTranspose1d(64, 1, kernel_size=(25,), stride=(4,), padding=(11,), output_padding=(1,))
    (10): Tanh()
  )
)
WaveDiscriminator(
  (dense): Linear(in_features=16384, out_features=1, bias=True)
  (seq): Sequential(
    (0): Conv1d(1, 64, kernel_size=(25,), stride=(4,), padding=(11,))
    (1): LeakyReLU(negative_slope=0.2, inplace=T